# Self-reflection through prompting
Stephan Raaijmakers, LUCL, 10.04.2025

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [ ]:
!pip install langchain_openai
from langchain_openai import ChatOpenAI

In [ ]:
from operator import itemgetter

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
llm_openai = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY
)

In [ ]:
llm_hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    #repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature=0.1
)

In [ ]:
rules = [
    'Mention diversity',
    "Mention discrimination",
    'No discussion of gun violence or harm',
]

In the templates below, specify which llm you want to use: llm_hf or llm_openai:

In [ ]:
template1 = """Answer the question:

Question: {query}
"""

prompt1 = ChatPromptTemplate.from_template(template1)

chain1 = (
    {
        "query": itemgetter("query"),
    }
    | prompt1
    | llm_openai
    | StrOutputParser()
)

template2 = """Based on the response {response} to {query}, evaluate which {rules} are violated

Question: {query}
Response: {response}
Rules:{rules}
"""

prompt2 = ChatPromptTemplate.from_template(template2)

chain2 = (
    {
        "query": itemgetter("query"),
        "response": itemgetter("response"),
        "rules": itemgetter("rules"),
    }
    | prompt2
    | llm_openai
    | StrOutputParser()
)


template3 = """Based on the evaluation {evaluation} of response {response} to {query}, reflect if the response {response} needs to be changed to satisfy the rules {rules} and why based on the query {query}"

Question: {query}
Response: {response}
Rules:{rules}
Evaluation:{evaluation}
"""

prompt3 = ChatPromptTemplate.from_template(template3)

chain3 = (
    {
        "query": itemgetter("query"),
        "response": itemgetter("response"),
        "rules": itemgetter("rules"),
        "evaluation": itemgetter("evaluation"),
    }
    | prompt3
    | llm_openai
    | StrOutputParser()
)





template4 = """Based on the reflection {reflection} of the evaluation {evaluation} of response {response} to {query}, reflect if the response {response} needs to be changed to satisfy the rules {rules} and why based on the query {query}"

Question: {query}
Response: {response}
Rules:{rules}
Evaluation:{evaluation}
Reflection:{reflection}
"""

prompt4 = ChatPromptTemplate.from_template(template4)

chain4 = (
    {
        "query": itemgetter("query"),
        "response": itemgetter("response"),
        "rules": itemgetter("rules"),
        "evaluation": itemgetter("evaluation"),
        "reflection": itemgetter("reflection"),
    }
    | prompt4
    | llm_openai # change to llm_hf if desired
    | StrOutputParser()
)





In [ ]:
# Set a query
query = 'How to win an election?'

In [ ]:
response=chain1.invoke({"query": query, "rules": rules})

In [ ]:
response

"Winning an election involves a combination of strategy, understanding the political landscape, effective communication, and grassroots organization. Here are some key steps to consider:\n\n1. **Understand the Political Landscape:**\n   - Research the demographics, key issues, and voting history of the area.\n   - Analyze the strengths and weaknesses of opponents.\n\n2. **Develop a Clear Message:**\n   - Craft a compelling and clear message that resonates with voters.\n   - Focus on issues that are important to the electorate.\n\n3. **Build a Strong Campaign Team:**\n   - Assemble a team of experienced and dedicated individuals.\n   - Ensure roles and responsibilities are clearly defined.\n\n4. **Fundraising:**\n   - Develop a robust fundraising strategy to support campaign activities.\n   - Engage with donors and host fundraising events.\n\n5. **Utilize Data and Analytics:**\n   - Use data to target voters effectively.\n   - Analyze voter behavior and tailor strategies accordingly.\n\

In [ ]:
evaluation=chain2.invoke({"query": query, "rules": rules,"response":response})

In [ ]:
evaluation

'The response does not explicitly mention diversity or discrimination, so it violates the "Mention diversity" and "Mention discrimination" rules. However, it does not discuss gun violence or harm, so it complies with the "No discussion of gun violence or harm" rule.'

In [ ]:
reflection=chain3.invoke({"query": query, "rules": rules,"response":response,"evaluation":evaluation})

In [ ]:
reflection

'To modify the response to satisfy the rules of "Mention diversity," "Mention discrimination," and "No discussion of gun violence or harm," the following adjustments can be made:\n\n1. **Understand the Political Landscape:**\n   - Research the demographics, key issues, and voting history of the area, paying close attention to diverse communities and their specific needs.\n   - Analyze the strengths and weaknesses of opponents, particularly how they address issues of diversity and discrimination.\n\n2. **Develop a Clear Message:**\n   - Craft a compelling and clear message that resonates with voters, emphasizing inclusivity and equal representation.\n   - Focus on issues that are important to the electorate, including policies that combat discrimination and promote diversity.\n\n3. **Build a Strong Campaign Team:**\n   - Assemble a team of experienced and dedicated individuals who reflect the diversity of the community.\n   - Ensure roles and responsibilities are clearly defined, with a

In [ ]:
final=chain4.invoke({"query": query, "rules": rules,"response":response,"evaluation":evaluation,"reflection":reflection})

In [ ]:
final

'To ensure that the response to "How to win an election?" aligns with the rules of "Mention diversity," "Mention discrimination," and "No discussion of gun violence or harm," here is a revised version:\n\n1. **Understand the Political Landscape:**\n   - Conduct thorough research on the demographics, key issues, and voting history of the area, with particular attention to the needs and concerns of diverse communities.\n   - Evaluate the strengths and weaknesses of opponents, especially their approaches to diversity and anti-discrimination policies.\n\n2. **Develop a Clear Message:**\n   - Create a compelling message that emphasizes inclusivity, equal representation, and the importance of combating discrimination.\n   - Prioritize issues that resonate with the electorate, including those that address and promote diversity and equality.\n\n3. **Build a Strong Campaign Team:**\n   - Assemble a team that reflects the community\'s diversity, ensuring a wide range of perspectives and ideas.\n